<a href="https://colab.research.google.com/github/DoubleCyclone/house-price-prediction/blob/main/notebooks/house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I will be working with the "**House Prices - Advanced Regression Techniques**" dataset today to perform
*   Exploratory Data Analysis
*   Data Preprocessing
*   Feature Engineering
*   Model Building
*   Evaluation
*   and Visualisation

First things first, I am going to mount google drive so that I can upload the dataset there and easily access it from the notebook.

The dataset (and the competition) is at [Kaggle Competition/Dataset Link](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview)

In [ ]:
from google.colab import drive

# Mount the Google Drive
drive.mount('/content/drive')

In [ ]:
# Let's import the other dependencies as well
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder, PowerTransformer
import warnings
warnings.filterwarnings('ignore')

# 1 - Exploratory Data Analysis
I will start by reading the train.csv which is the training dataset and displaying a small portion of it.

In [ ]:
# Load the training and test datasets, examine their shapes and contents
data_train = pd.read_csv('/content/drive/MyDrive/Colab_Materials/House_Price_Estimation/train.csv')
data_test = pd.read_csv('/content/drive/MyDrive/Colab_Materials/House_Price_Estimation/test.csv')
# I will drop the ID columns as they are not used for training models
data_train = data_train.drop('Id', axis=1)
data_test = data_test.drop('Id', axis=1)

print(f"Shape of the train dataset = {data_train.shape}")
data_train.head()

In [ ]:
print(f"Shape of the test dataset = {data_test.shape}")
data_test.head()

Seems like there are almost an equal amount of data in both datasets. There is also one more column in the train dataset called **SalePrice** is the sale price which will be the labels the model will learn from in this case. How about getting an idea of its distribution in the dataset?

In [ ]:
# Use the pandas.describe method to automatically generate valuable information about the dataset (only for a numerical variable in this case)
print(data_train['SalePrice'].describe())

# Plot the distribution of the SalePrice column
plt.figure(figsize=(5, 5))
sns.distplot(data_train['SalePrice'], color='g', bins=100, hist_kws={'alpha': 0.4});

# Move the SalePrice (label)
data_y = data_train['SalePrice']

# Drop label from the DataFrame
data_train.drop('SalePrice', axis=1, inplace=True)

By looking at the output of the pandas.describe function and the graph, we can see that most of the Sale Prices reside at around 180000 and the standard deviation is quite low. Meaning that the data has low variability. How do I decide if the standard deviation is low or not? Firstly, I calculate the range of the data which is **max - min**. In this case **755000 - 34900 = 720100**. If the standard deviation is close to the range, I can say that the variability is high but in our case, standard deviation is approximately 10x lesser than the range which lets us conclude that the standard deviation, thus the variability in Sale Prices is low. <br><br>
Now let's see what type of data is stored in the training dataset.

In [ ]:
# Print the list of unique data types
list(set(data_train.dtypes.tolist()))

Now let's store the numerical data in a new DataFrame so that we can use it easily.

In [ ]:
# Create a new DataFrame for numerical data only
train_num = data_train.select_dtypes(include = ['float64', 'int64'])
train_num.head()

While at that, let's plot the distributions of all these numerical features at the same time.

In [ ]:
train_num.hist(figsize=(20, 20), bins=50, xlabelsize=8, ylabelsize=8)

I considered removing some of these columns if they could be unrelated to a house's sale price but I think all of these are useful so they will be kept. Let's take a look at the categorical features this time.

In [ ]:
# Create a new DataFrame for categorical data only
train_cat = data_train.select_dtypes(include = ['O'])
train_cat.head()

Seems like most of our features are categorical. Let's plot their distributions as well.

In [ ]:
# Create a composed plot with matplotlib.subplots()
fig, axs = plt.subplots(8, 6, figsize=(24, 32))

# Flatten the axes so that iterating is easier
axs = axs.flatten()

# Iterate through each feature/column and plot their countplots
for i, col in enumerate(train_cat.columns):
  sns.countplot(x=train_cat[col], ax=axs[i])
  axs[i].tick_params(axis='x', rotation=90)

# Use matplotlib.tight_layout() to prevent overlapping
fig.tight_layout()


As it is not desirable to directly enumerate categorical values. For example, it does not make sense to give paved a value of 0 and gravel a value of 1 for the **Street** feature. We would rather use an approach called **One-Hot Encoding** which is basically to give every unique entry its own binary value like;
* Paved = 1
* Gravel = 0

so that this means there is a paved street and not gravel around the house. It should be noted that only one of these values can be 1 (True) at a time but all can be 0 (False) if that entry is missing. <br><br>
Also, numerical features' ranges vary by a large margin in the dataset. If we take a look at the features like **FullBath** which is a number of full bathrooms above grade and **TotalLotArea** which can get a value up to 200000~ as can be seen from the plots. So we should definitely normalize the numerical features so that we end up with values that are standardized. For example, limiting their range to 0-1 interval. <br>
This
* helps models converge faster
* prevents the **NaN** trap which is when a value exceeds the floating point precision limit so that eventually every number in the model became **NaN**

There are a few normalization methods in my mind so let's try to decide on which one would be the best fit for our dataset.
- **Linear Scaling** : Best when
  - there are few or no outliers, and the outliers aren't extreme.
  - distribution of the data is approximately uniform.
- **Z-Score Scaling** : Best when
  - the distribution resembles a normal distribution or something close to it.
- **Log Scaling** : Best when
  - low values of x have very high values of y
  - as the values of x increase, values of y quickly decrease
  
I might use something completely different who knows...

# 2 - Data Preprocessing
This has been a huge information dump but lets just examine what portion of each feature is missing now to see if we want to remove anything.

In [ ]:
# Use the DataFrame.isna().sum() with a condition to get all the counts of nan values for every column that has at least one, then turn it into a percentage
train_num.isna().sum()[data_train.isna().sum() > 0] / len(data_train) * 100

In [ ]:
# Do the same with categorical features
train_cat.isna().sum()[data_train.isna().sum() > 0] / len(data_train) * 100

Turns out most of the features that can take the value of **NaN** are categorical and it makes sense. A house does not necessarily have to have Pool for example so it can have a value of 0 for each **One-Hot Encoded** feature. Even the numerical data makes sense as if a property does not have a garage for example, it should not have a **GarageYrBlt** value. Though, this is a problem as we do not want any **NaN** values in our dataset and I am going to only scale the numerical ones so the **NaN** values are not going to disappear. To remedy this, I will
- Take the median for LotFrontage
- Set MasVnrArea to 0
- Set GarageYrBlt to the same as YearBuilt

hopefully to get rid of those in the most reasonable way possible.<br><br>

As it is the easiest part, let's turn categorical features into One-Hot Encodings.


In [ ]:
# Handle missing data
train_num['LotFrontage'] = train_num['LotFrontage'].fillna(train_num['LotFrontage'].median())
train_num['MasVnrArea'] = train_num['MasVnrArea'].fillna(0)
train_num['GarageYrBlt'] = train_num['GarageYrBlt'].fillna(train_num['YearBuilt'])

Now let's take **One-Hot-Encode** the categorical features.

In [ ]:
# Get the categorical columns
categorical_columns = train_cat.columns.tolist()

# Specify encoder
encoder = OneHotEncoder(sparse_output=False)

# Create a Tensor for the one-hot-encoded features
one_hot_encoded = encoder.fit_transform(data_train[categorical_columns])

# Create a DataFrame from the Tensor with the appropriate column names
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Display a portion of it
one_hot_df.head()

Now we have 267 columns of **True** or **False** values for categorical features. For the next step, let's try to use Scikit-Learn's PowerTransformer to normalize the data instead of the other three methods as I don't think any of them fit well to all (or even most) features.

In [ ]:
# Initialize the PowerTransformer
pt = PowerTransformer(method='yeo-johnson', standardize=True)

# Transform only the numerical the data
train_num_normalized = pd.DataFrame(pt.fit_transform(train_num), columns=pt.get_feature_names_out(train_num.columns))

# Plot them after the transformation
train_num_normalized.hist(figsize=(20, 20), bins=50, xlabelsize=8, ylabelsize=8)

These distributions look a lot better than how they were originally. The scale problem is solved as all the numerical features now span a similar range. Finally let's create the actual training dataset that we are going to use to train models.

In [ ]:
# Concatenate the scaled numerical features and one-hot-encoded categorical values
data_train_normalized = pd.concat([train_num_normalized, one_hot_df], axis=1)

# Display a portion of it
data_train_normalized.head()

Now that I have the **normalized** and **one-hot-encoded** features. I will convert this DataFrame into a **PyTorch Tensor** so that we can start use it for training easily. Let's convert the labels as well.

In [ ]:
# Convert DataFrame to a PyTorch Tensor
X_train = torch.tensor(data_train_normalized.values, dtype=torch.float32)

# Display the shape of the tensors
print(f"Training Data Shape : {X_train.shape}")

Now that our data is ready to go. I will initialize the Linear Regression Model with **PyTorch**. I will be using **RMSE (Root Mean Squared Error)** as the loss function to abide by the competition rules. Turns out the loss is calculated between the logarithms of the Sale Prices so I will need to transform the label DataFrame. I will also use the **AdamW** optimizer.

In [ ]:
class LinearRegressionModel(torch.nn.Module):

    def __init__(self, input_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(input_dim, 1)

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [ ]:
# Create RMSE Loss (MSE but inside a square root)
class RMSELoss(torch.nn.Module):
    def __init__(self, eps = 1e-6):
        super(RMSELoss,self).__init__()
        self.eps = eps

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y) + self.eps)
        return loss

In [ ]:
# Model initialization
lr_model = LinearRegressionModel(input_dim=X_train.shape[1])

# Select Loss Function and optimizer
criterion = RMSELoss(eps=1e-6)
optimizer = torch.optim.AdamW(lr_model.parameters(), lr=0.01, weight_decay=0.01)

In [ ]:
# Transform the labels by calculating their logarithms
data_y_log = np.log1p(data_y)

# Transform the labels to a tensor
y_train = torch.tensor(data_y_log.values, dtype=torch.float32).view(-1, 1)
print(f"Training Labels Shape : {y_train.shape}")

Okay. Everything should be ready to start our first model training. As I have few data (Yes 1460 entries is few...), I think training for 150 epochs should be reasonable. Don't forget that training for too long has the risk of overfitting the model.

In [ ]:
# To track losses
losses = []

for epoch in range(150):

    # Perform a forward pass (prediction)
    pred_y = lr_model(X_train)

    # Compute and print loss
    loss = criterion(pred_y, y_train)

    # Track losses
    losses.append(loss.item())

    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"epoch {epoch + 1}, loss {loss.item()}")

# Plot loss function
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('RMSE Loss')
plt.title('Training Loss')
plt.show()

Our loss function looks pretty good, it converges well. There is an oscillation between epochs ~24-75 but it stabilizes after that.

In [ ]:
# We aren't going to update the model so we use no_grad()
with torch.no_grad():
    predictions = lr_model(X_train).numpy()

plt.scatter(y_train.numpy(), predictions, alpha=0.5)
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--')  # perfect line
plt.xlabel('Actual (log price)')
plt.ylabel('Predicted (log price)')
plt.title('Predictions vs Actuals')
plt.show()

Points hug the line well enough. The correlation looks strong except for the slight spread at lower prices.

In [ ]:
residuals = y_train.numpy() - predictions

plt.scatter(predictions, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.title('Residual Plot')
plt.show()

The points are randomly scattered around 0 and there is no obvious pattern which is a good indicator that the model is learning the relationships well.